In [1]:
import pymysql.cursors
import pandas as pd
from pandas.io import sql
from sqlalchemy import create_engine
from sqlalchemy.sql import text

In [2]:
connect_string = 'mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8mb4'.format(
    'root', 'password', 'localhost', '3306', 'rec_engine')
engine = create_engine(connect_string, convert_unicode = True, echo = False)
con = engine.connect()

In [8]:
item = pd.read_csv('item.csv')
agg = pd.read_csv('agg.csv')

In [9]:
item.head()

,item
0,aluminum foil
1,apples
2,baby items
3,bagels
4,bananas


In [10]:
agg.head()

,Item_name,user_id,count
0,aluminum foil,47,1
1,aluminum foil,113,1
2,aluminum foil,527,1
3,aluminum foil,553,2
4,aluminum foil,613,1


In [11]:
item.to_sql(name = 'ITEM', con = con, if_exists = 'append', index = False)

In [12]:
agg.to_sql(name = 'AGG', con = con, if_exists = 'append', index = False)

In [3]:
# running the recommendation search for user-18710
def exec_query():
    with engine.connect() as con:
        result = con.execute(
            'SELECT item.item AS Recommendation, count(1) AS Frequency ' +
            'FROM item, agg, (SELECT agg3.item_name, agg3.user_id ' +
            'FROM agg agg, agg agg2, agg agg3 '+
            'WHERE agg.user_id = "18710" ' +
            'AND agg.item_name = agg2.item_name ' +
            'AND agg2.user_id != "18710" ' +
            'AND agg3.user_id = agg2.user_id ' +
            'AND agg3.user_id NOT IN ( ' +
                'SELECT DISTINCT item_name ' +
                'FROM  agg agg ' +
                'WHERE agg.user_id = "18710") ' +
            ') recommended_products ' +
            'WHERE agg.item_name = item.item ' +
            'AND agg.item_name IN (recommended_products.item_name) ' +
            'AND agg.user_id = recommended_products.user_id ' +
            'GROUP BY item.item ' +
            'ORDER BY Frequency desc;'
        )

In [4]:
for i in range(10):
    %time exec_query()

CPU times: user 3.94 ms, sys: 2.3 ms, total: 6.23 ms
Wall time: 26.3 s
CPU times: user 2.84 ms, sys: 661 µs, total: 3.5 ms
Wall time: 26.4 s
CPU times: user 2.21 ms, sys: 594 µs, total: 2.8 ms
Wall time: 27 s
CPU times: user 2.43 ms, sys: 582 µs, total: 3.01 ms
Wall time: 26.9 s
CPU times: user 2.58 ms, sys: 678 µs, total: 3.26 ms
Wall time: 28.2 s
CPU times: user 2.24 ms, sys: 565 µs, total: 2.81 ms
Wall time: 26 s
CPU times: user 2.1 ms, sys: 514 µs, total: 2.62 ms
Wall time: 26.5 s
CPU times: user 1.98 ms, sys: 463 µs, total: 2.45 ms
Wall time: 26.7 s
CPU times: user 1.98 ms, sys: 476 µs, total: 2.46 ms
Wall time: 26.3 s
CPU times: user 2.11 ms, sys: 587 µs, total: 2.7 ms
Wall time: 26.9 s
